<a href="https://colab.research.google.com/github/shwoa/Capstone/blob/main/Pycaret_%EA%B2%B0%EC%B8%A1%EC%B9%98%EC%A0%9C%EA%B1%B0_GWAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pycaret GWAS 결측치 제거**

*  입력 파일 :


1. 유전형인코딩_결측치_행_제거.csv
2. 2016년 표현형 데이터.xlsx
3. GWAS_SNP(N).csv

In [ ]:
# ✅ 라이브러리 설치
!pip install pycaret lightgbm xgboost catboost openpyxl --quiet

In [ ]:
# ✅ 라이브러리
from pycaret.regression import setup, compare_models, pull
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# ✅ Colab에 파일 업로드
from google.colab import files
uploaded = files.upload()

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x79341dbea980>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 1187, in _make_controller_from_path
    lib_controller = controller_class(
                     ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: dlopen() error


Saving 2016년 표현형 데이터.xlsx to 2016년 표현형 데이터.xlsx
Saving GWAS_SNP(100).csv to GWAS_SNP(100).csv
Saving GWAS_SNP(500).csv to GWAS_SNP(500).csv
Saving GWAS_SNP(1000).csv to GWAS_SNP(1000).csv
Saving GWAS_SNP(3000).csv to GWAS_SNP(3000).csv
Saving GWAS_SNP(5000).csv to GWAS_SNP(5000).csv
Saving GWAS_SNP(10000).csv to GWAS_SNP(10000).csv
Saving 유전형인코딩_결측치_행_제거.csv to 유전형인코딩_결측치_행_제거.csv


In [ ]:
# ✅ 파일 경로 지정 (⬅️ 업로드한 파일 이름으로 변경)
geno_path = "유전형인코딩_결측치_행_제거.csv"
pheno_path = "2016년 표현형 데이터.xlsx"

snp_files = {
    "Top 100": "GWAS_SNP(100).csv",
    "Top 500": "GWAS_SNP(500).csv",
    "Top 1000": "GWAS_SNP(1000).csv",
    "Top 3000": "GWAS_SNP(3000).csv",
    "Top 5000": "GWAS_SNP(5000).csv",
    "Top 10000": "GWAS_SNP(10000).csv"
}

In [ ]:
# ✅ 데이터 로딩
geno_df = pd.read_csv(geno_path, index_col=0)
pheno_df = pd.read_excel(pheno_path)
pheno_df.columns = pheno_df.columns.str.strip()
geno_df_t = geno_df.T
geno_df_t.index.name = "Genotype"
pheno_df.set_index("Genotype", inplace=True)

# ✅ 성능 비교 결과 저장 리스트
all_results = []

In [ ]:
# ✅ 표현형별 + SNP 수별 분석 루프
trait_map = {
    "과중": "과중 (g)",
    "과장": "과장 (mm)",
    "과폭": "과폭 (mm)",
    "과피두께": "과피두께 (mm)",
    "당도": "당도 (%)",
    "과실경도": "과실경도 (kg)"
}

for label, file_path in snp_files.items():
    snp_df = pd.read_csv(file_path)
    for trait in snp_df["Trait"].unique():
        try:
            actual_trait = trait_map.get(trait)
            if actual_trait not in pheno_df.columns:
                raise ValueError(f"{trait} → {actual_trait} 가 표현형에 없음")

            top_snps = snp_df[snp_df["Trait"] == trait]["SNP"].tolist()
            available_snps = [snp for snp in top_snps if snp in geno_df.index]

            if len(available_snps) < 5:
                raise ValueError(f"{trait} - {label}: 사용 가능한 SNP가 너무 적습니다.")

            geno_subset = geno_df.loc[available_snps].T
            geno_subset["target"] = pheno_df[actual_trait]

            setup(data=geno_subset, target="target", session_id=42, use_gpu=False, verbose=False)
            compare_models(include=["rf", "xgboost", "lightgbm", "catboost"], n_select=4)
            result_df = pull()
            result_df["SNP 개수"] = label
            result_df["표현형"] = actual_trait

            print(f"✅ 표현형: {actual_trait} | SNP 수: {label} | 사용된 SNP 수: {len(available_snps)}")
            display(result_df)

            all_results.append(result_df)

        except Exception as e:
            print(f"⚠️ 오류 - 표현형: {trait}, SNP: {label}, 원인: {e}")
            continue


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,29.4328,1796.4126,41.7009,0.3360,0.5443,0.5975,0.1810
rf,Random Forest Regressor,31.0529,1998.2405,44.0941,0.2273,0.5696,0.6115,0.5350
xgboost,Extreme Gradient Boosting,32.3824,2092.1826,45.0892,0.1836,0.5986,0.6103,0.0590
catboost,CatBoost Regressor,32.6564,2092.4926,45.2557,0.1771,0.5987,0.6331,0.4500


Processing:   0%|          | 0/24 [00:00<?, ?it/s]

✅ 표현형: 과중 (g) | SNP 수: Top 100 | 사용된 SNP 수: 28


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
lightgbm,Light Gradient Boosting Machine,29.4328,1796.4126,41.7009,0.3360,0.5443,0.5975,0.181,Top 100,과중 (g)
rf,Random Forest Regressor,31.0529,1998.2405,44.0941,0.2273,0.5696,0.6115,0.535,Top 100,과중 (g)
xgboost,Extreme Gradient Boosting,32.3824,2092.1826,45.0892,0.1836,0.5986,0.6103,0.059,Top 100,과중 (g)
catboost,CatBoost Regressor,32.6564,2092.4926,45.2557,0.1771,0.5987,0.6331,0.450,Top 100,과중 (g)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,6.3670,78.3271,8.5585,0.2733,0.1766,0.1428,0.0490
rf,Random Forest Regressor,6.7259,91.3648,9.3376,0.1085,0.1917,0.1518,0.1030
catboost,CatBoost Regressor,7.1301,104.5211,9.9601,-0.0310,0.2044,0.1625,0.3180
xgboost,Extreme Gradient Boosting,7.6719,118.8421,10.5754,-0.1778,0.2171,0.1737,0.0380


...

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
catboost,CatBoost Regressor,0.0774,0.0100,0.0980,0.3248,0.0611,0.1358,23.200,Top 10000,과실경도 (kg)
rf,Random Forest Regressor,0.0786,0.0104,0.0994,0.2983,0.0621,0.1382,1.332,Top 10000,과실경도 (kg)
xgboost,Extreme Gradient Boosting,0.0852,0.0124,0.1069,0.1768,0.0670,0.1502,1.149,Top 10000,과실경도 (kg)
lightgbm,Light Gradient Boosting Machine,0.0853,0.0120,0.1073,0.1694,0.0670,0.1511,1.841,Top 10000,과실경도 (kg)


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
catboost,CatBoost Regressor,0.6224,0.6745,0.8127,0.3597,0.1565,0.1564,29.0870
rf,Random Forest Regressor,0.6394,0.7080,0.8229,0.3174,0.1591,0.1626,1.5020
xgboost,Extreme Gradient Boosting,0.6408,0.7309,0.8265,0.2893,0.1582,0.1613,1.3720
lightgbm,Light Gradient Boosting Machine,0.6933,0.7907,0.8793,0.2506,0.1658,0.1745,1.1420


Processing:   0%|          | 0/24 [00:00<?, ?it/s]

✅ 표현형: 당도 (%) | SNP 수: Top 10000 | 사용된 SNP 수: 2096


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
catboost,CatBoost Regressor,0.6224,0.6745,0.8127,0.3597,0.1565,0.1564,29.087,Top 10000,당도 (%)
rf,Random Forest Regressor,0.6394,0.7080,0.8229,0.3174,0.1591,0.1626,1.502,Top 10000,당도 (%)
xgboost,Extreme Gradient Boosting,0.6408,0.7309,0.8265,0.2893,0.1582,0.1613,1.372,Top 10000,당도 (%)
lightgbm,Light Gradient Boosting Machine,0.6933,0.7907,0.8793,0.2506,0.1658,0.1745,1.142,Top 10000,당도 (%)


In [ ]:
# ✅ 결과 저장
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    final_df.to_csv("모델성능비교_GWAS_전체정리.csv", index=False)
    print("📦 결과 저장 완료 → 모델성능비교_GWAS_전체정리.csv")
    display(final_df.head(10))
else:
    print("❌ 저장할 결과가 없습니다.")

📦 결과 저장 완료 → 모델성능비교_GWAS_전체정리.csv


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
0,Light Gradient Boosting Machine,29.4328,1796.4126,41.7009,0.3360,0.5443,0.5975,0.181,Top 100,과중 (g)
1,Random Forest Regressor,31.0529,1998.2405,44.0941,0.2273,0.5696,0.6115,0.535,Top 100,과중 (g)
2,Extreme Gradient Boosting,32.3824,2092.1826,45.0892,0.1836,0.5986,0.6103,0.059,Top 100,과중 (g)
3,CatBoost Regressor,32.6564,2092.4926,45.2557,0.1771,0.5987,0.6331,0.450,Top 100,과중 (g)
4,Light Gradient Boosting Machine,6.3670,78.3271,8.5585,0.2733,0.1766,0.1428,0.049,Top 100,과장 (mm)
5,Random Forest Regressor,6.7259,91.3648,9.3376,0.1085,0.1917,0.1518,0.103,Top 100,과장 (mm)
6,CatBoost Regressor,7.1301,104.5211,9.9601,-0.0310,0.2044,0.1625,0.318,Top 100,과장 (mm)
7,Extreme Gradient Boosting,7.6719,118.8421,10.5754,-0.1778,0.2171,0.1737,0.038,Top 100,과장 (mm)
8,CatBoost Regressor,8.5195,124.7849,10.9169,0.4355,0.2113,0.1812,0.336,Top 100,과폭 (mm)
9,Extreme Gradient Boosting,8.6978,132.1894,11.1596,0.4149,0.2192,0.1848,0.036,Top 100,과폭 (mm)


In [ ]:
import pandas as pd

# 1. 기존 결과 불러오기
df = pd.read_csv("모델성능비교_GWAS_전체정리.csv")

# 2. 표현형 + SNP 수별 R² 최고 모델만 추출
ranking_df = df.loc[df.groupby(["표현형", "SNP 개수"])["R2"].idxmax()]

# 3. 결과 저장
ranking_df.to_csv("모델성능_GWAS_랭킹요약.csv", index=False)

# 4. 미리보기
ranking_df.head()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
16,Light Gradient Boosting Machine,0.0829,0.0109,0.1035,0.2201,0.0644,0.1432,0.170,Top 100,과실경도 (kg)
64,Random Forest Regressor,0.0799,0.0114,0.1043,0.2204,0.0651,0.1387,0.178,Top 1000,과실경도 (kg)
136,CatBoost Regressor,0.0774,0.0100,0.0980,0.3248,0.0611,0.1358,23.200,Top 10000,과실경도 (kg)
88,Light Gradient Boosting Machine,0.0793,0.0104,0.0993,0.2884,0.0618,0.1374,0.845,Top 3000,과실경도 (kg)
40,CatBoost Regressor,0.0780,0.0109,0.1018,0.2576,0.0635,0.1355,0.850,Top 500,과실경도 (kg)


In [ ]:
# 📌 Colab 전용 성능 요약 코드 (R² 기준 요약 3종)

import pandas as pd

# # 👉 파일 업로드
# from google.colab import files
# uploaded = files.upload()  # 모델성능비교_GWAS_전체정리.csv 업로드

# 👉 파일 로드
df = pd.read_csv("모델성능비교_GWAS_전체정리.csv")

# ✅ 1. 표현형별 최고 R² 모델 추출
top_r2_by_trait = df.loc[df.groupby("표현형")["R2"].idxmax()].reset_index(drop=True)

# ✅ 2. SNP 개수별 평균 R² 계산
mean_r2_by_snp = df.groupby("SNP 개수")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# ✅ 3. 모델별 평균 R² 계산
mean_r2_by_model = df.groupby("Model")["R2"].mean().reset_index().rename(columns={"R2": "평균 R²"})

# 👉 결과 출력
print("📌 표현형별 최고 R² 요약")
display(top_r2_by_trait)

print("📌 SNP 개수별 평균 R²")
display(mean_r2_by_snp)

print("📌 모델별 평균 R²")
display(mean_r2_by_model)

📌 표현형별 최고 R² 요약


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),SNP 개수,표현형
0,CatBoost Regressor,0.0774,0.0100,0.0980,0.3248,0.0611,0.1358,23.200,Top 10000,과실경도 (kg)
1,Random Forest Regressor,6.1160,76.6475,8.4148,0.3170,0.1765,0.1423,1.700,Top 10000,과장 (mm)
2,Random Forest Regressor,26.7228,1415.1064,37.2610,0.4466,0.5183,0.5978,0.850,Top 5000,과중 (g)
3,Light Gradient Boosting Machine,7.6064,102.7725,10.0916,0.4943,0.2031,0.1676,1.295,Top 5000,과폭 (mm)
4,Random Forest Regressor,0.5292,0.5380,0.7183,0.1898,0.1265,0.1185,1.366,Top 10000,과피두께 (mm)
5,CatBoost Regressor,0.6224,0.6745,0.8127,0.3597,0.1565,0.1564,29.087,Top 10000,당도 (%)


📌 SNP 개수별 평균 R²


,SNP 개수,평균 R²
0,Top 100,0.187621
1,Top 1000,0.140896
2,Top 10000,0.279467
3,Top 3000,0.234537
4,Top 500,0.147275
5,Top 5000,0.242171


📌 모델별 평균 R²


,Model,평균 R²
0,CatBoost Regressor,0.221978
1,Extreme Gradient Boosting,0.111192
2,Light Gradient Boosting Machine,0.234942
3,Random Forest Regressor,0.253200
